In [1]:
import sqlite3 as sql
import pandas as pd 
import numpy as np

conn = sql.connect('NEST_Pandemia.sqlite') 

queryall = 'select * from nest limit 1' 
df_titulos = pd.read_sql_query(queryall, conn)

In [2]:
# Writers 
# escribe excel file para DP (0001,0002)  
writer1 = pd.ExcelWriter("lesiones_dp_univariados.xlsx")
# escribe excel file para DP (0004,0005)
writer2 = pd.ExcelWriter("colision_comprensivo_univariados.xlsx")
# escribe excel file cobertura asistencia medica
writer3 = pd.ExcelWriter("asistenciamedica_univariados.xlsx")  

In [3]:
# lista de coberturas por daños propios y asistencia medica 
coberturas_cc = ["0004", "0005"]
coberturas_am = ["0003"]

# pedazo de query que quiero que sea para que la columna del indice sea la que yo quiero 
buscadores = ["strftime('%Y', FECINIVALID )", \
              "strftime('%Y', FECINIVALID ) - anoveh", \
              "strftime('%Y', FECINIVALID ) - strftime('%Y', FECNAC )", \
              "(DESCMARCA)", \
              "(DESCMARCA || '_' || DESCMODELO)",
              "GRUPORELATIVIDAD", \
              "SEXO" 
            ]
# da un titulo a cada buscador para el sheet de excel, más facil de entender el query 
titulo_buscadores = ["ANOEMISION", "EDADVEH", "EDADCONDUCTOR", "MARCA", "MARCA_MODELO", "GRUPORELATIVIDAD", "SEXO"]

In [4]:
def querys(coberturas, buscadores, writer):
    for c in coberturas:
        # initialize position
        y = 0 
        # loop through each buscador
        for b in buscadores:
            # sql query para select columnas de NEST que se necesitan para el univariado
            query = "select "+b+" agrupador, \
                sum(Dias_Expuesto)/365 anos_expuestos, \
                sum(ACT_"+c+") accidentes, \
                sum(cl_"+c+") importe, \
                sum(sa_"+c+") sum_aseg \
                    from nest \
                    where tipoveh= 'P' and \
                    tipopdcion ='P' and \
                FECINIVALID > '2010-01-01' and \
                fecinivalid < '2023-01-01'\
                    group by agrupador" 

            # print(query)
            # Read the query results into a pandas DataFrame
            df_variables= pd.read_sql_query(query, conn)

            # agregando columnas
            df_variables['frecuencia'] = df_variables['accidentes'] / df_variables['anos_expuestos']
            df_variables['coste_medio'] = df_variables['importe'] / df_variables['accidentes']
            df_variables['sum_aseg_media'] = df_variables['sum_aseg']/ df_variables['anos_expuestos']
            df_variables['tasa_riesgo'] = df_variables['importe'] / df_variables['sum_aseg']

            df_variables.to_excel(writer, sheet_name= c+"_"+titulo_buscadores[y])
            # Increment the position for buscadores
            
            y += 1




In [5]:
# Usar function "querys" para hacer excels de 0004 y 0005 y 0003, respectivamente
querys(coberturas_cc, buscadores, writer2)
querys(coberturas_am, buscadores, writer3)


In [6]:
coberturas = [["0001", "0107"],["0002", "0108"]]

for c in coberturas:
    # loop through each buscador
    for y, b in enumerate(buscadores):
        # sql query para select columnas de NEST que se necesitan para el univariado 
        query = "SELECT agrupador, anos_expuestos, \
                SUM(accidentes) AS accidentes,\
                SUM(importe) AS importe,SUM(sum_aseg) AS sum_aseg \
                FROM (\
                    SELECT \
                    "+b+" agrupador, \
                    sum(Dias_Expuesto)/365 anos_expuestos, \
                    sum(ACT_"+c[0]+") accidentes,\
                    sum(cl_"+c[0]+") importe, \
                    sum(sa_"+c[0]+") sum_aseg \
                    FROM nest\
                    WHERE tipoveh = 'P' AND FECINIVALID > '2010-01-01' AND fecinivalid < '2023-01-01' \
                    GROUP BY agrupador \
                UNION ALL \
                    SELECT \
                    "+b+" agrupador, \
                    sum(Dias_Expuesto)/365 anos_expuestos, \
                    sum(ACT_"+c[1]+") accidentes,\
                    sum(cl_"+c[1]+") importe, \
                    sum(sa_"+c[1]+") sum_aseg \
                    FROM nest\
                    WHERE tipoveh = 'P' AND FECINIVALID > '2010-01-01' AND fecinivalid < '2023-01-01'\
                    GROUP BY agrupador \
                ) combined_results \
                GROUP BY agrupador, anos_expuestos;"
    
        # print(query)
        
        # Read the query results into a pandas DataFrame
        df_lesiones = pd.read_sql_query(query, conn)
        # agregando columnas
        df_lesiones['frecuencia'] = df_lesiones['accidentes'] / df_lesiones['anos_expuestos']
        df_lesiones['coste_medio'] = df_lesiones['importe'] / df_lesiones['accidentes']
        df_lesiones['sum_aseg_media'] = df_lesiones['sum_aseg']/ df_lesiones['anos_expuestos']
        df_lesiones['tasa_riesgo'] = df_lesiones['importe'] / df_lesiones['sum_aseg']
        df_lesiones.to_excel(writer1, sheet_name= c[0]+"_"+titulo_buscadores[y])


In [ ]:
# Save Excel files
writer1.save()
writer2.save()
writer3.save()